In [2]:
import modern_robotics as mr
import numpy as np

np.set_printoptions(suppress=True)
from dataclasses import dataclass
from tpk4170.visualization import Ur5Visualizer

In [4]:
v = Ur5Visualizer()

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [4]:
class Ur5:
    W1 = 0.10915
    W2 = 0.0823
    L1 = 0.425
    L2 = 0.39225
    H1 = 0.089159
    H2 = 0.09465

    def __init__(self):
        self._M = np.array(
            [
                [-1.0, 0.0, 0.0, self.L1 + self.L2],
                [0.0, 0.0, 1.0, self.W1 + self.W2],
                [0.0, 1.0, 0.0, self.H1 - self.H2],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )

        self._SList = np.array(
            [
                [0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
                [0.0, 1.0, 0.0, -self.H1, 0.0, 0.0],
                [0.0, 1.0, 0.0, -self.H1, 0.0, self.L1],
                [0.0, 1.0, 0.0, -self.H1, 0.0, self.L1 + self.L2],
                [0.0, 0.0, -1.0, -self.W1, self.L1 + self.L2, 0.0],
                [0.0, 1.0, 0.0, self.H2 - self.H1, 0.0, self.L1 + self.L2],
            ]
        ).T

    def fkin(self, theta):
        return mr.FKinSpace(self._M, self._SList, theta)


PI_HALF = np.pi / 2
PI = np.pi

In [5]:
ur5 = Ur5()

In [6]:
theta = [0.0,0.0,0.0,0.0,0.0,0.0]
ur5.fkin(theta)

array([[-1.      ,  0.      ,  0.      ,  0.81725 ],
       [ 0.      ,  0.      ,  1.      ,  0.19145 ],
       [ 0.      ,  1.      ,  0.      , -0.005491],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [7]:
D1 = Ur5.H1
A2 = Ur5.L1
A3 = Ur5.L2
D4 = Ur5.W1
D5 = Ur5.H2
D6 = Ur5.W2

In [8]:
@dataclass
class DHLink:
    a: float
    alpha: float
    d: float
    theta: float

    def matrix(self) -> np.array:
        ct = np.cos(self.theta)
        st = np.sin(self.theta)
        ca = np.cos(self.alpha)
        sa = np.sin(self.alpha)
        return np.array(
            [
                [ct, -st * ca, st * sa, self.a * ct],
                [st, ct * ca, -ct * sa, self.a * st],
                [0.0, sa, ca, self.d],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )

In [9]:
l1 = DHLink(0.0, PI_HALF, D1, PI)
l2 = DHLink(-A2, 0.0, 0.0, 0.0)
l3 = DHLink(-A3, 0.0, 0.0, 0.0)
l4 = DHLink(0.0, PI_HALF, D4, 0.0)
l5 = DHLink(0.0, -PI_HALF, D5, 0.0)
l6 = DHLink(0.0, 0.0, D6, 0)

In [10]:
fk_dh_zero = (
    l1.matrix()
    @ l2.matrix()
    @ l3.matrix()
    @ l4.matrix()
    @ l5.matrix()
    @ l6.matrix()
)
print(fk_dh_zero)

[[-1.       -0.        0.        0.81725 ]
 [ 0.       -0.        1.        0.19145 ]
 [ 0.        1.        0.       -0.005491]
 [ 0.        0.        0.        1.      ]]


In [11]:
theta = [0.0,0.0,0.0,0.0,0.0,0.0]
ur5.fkin(theta)

array([[-1.      ,  0.      ,  0.      ,  0.81725 ],
       [ 0.      ,  0.      ,  1.      ,  0.19145 ],
       [ 0.      ,  1.      ,  0.      , -0.005491],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [37]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from pythreejs import Object3D
from transformations import quaternion_from_matrix

from pythreejs import ArrowHelper as Arrow

In [40]:
class DHFrame(Axes):
    def __init__(self, trf):
        Axes.__init__(self, 0.1)
        self.quaternion = np.roll(quaternion_from_matrix(trf), -1).tolist()
        self.position = (trf[:3, 3]).tolist()

In [41]:
class ScrewAxis(Arrow):
    def __init__(self, direction, origin, length=0.2 , color=0xffff00):
        Arrow.__init__(self, direction, origin, hex=color)
        

In [43]:
viewer = Viewer(background="white")

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [45]:
viewer.add(Grid())

In [49]:
# example add an S axis
viewer.add(ScrewAxis([0,1,1], [-.1,0,.2]))

In [27]:
T01 = l1.matrix()
T02 = l1.matrix() @ l2.matrix()
T03 = l1.matrix() @ l2.matrix() @ l3.matrix()
T04 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix()
T05 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix() @ l5.matrix()
T06 = l1.matrix() @ l2.matrix() @ l3.matrix() @ l4.matrix() @ l5.matrix() @ l6.matrix()

In [28]:
viewer.add(Axes(0.1))

In [29]:
viewer.add(DHFrame(T01))

In [30]:
viewer.add(DHFrame(T02))

In [31]:
viewer.add(DHFrame(T03))

In [32]:
viewer.add(DHFrame(T04))

In [33]:
viewer.add(DHFrame(T05))

In [34]:
viewer.add(DHFrame(T06))